In [106]:
import warnings
warnings.filterwarnings("ignore")

# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [107]:
!pwd

/mnt/x/1_projects/dsi/scaling_to_production/02_activities/assignments


In [108]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [109]:
X = adult_dt.drop('income', axis=1)
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [110]:
Y = adult_dt['income']
pd.DataFrame(Y).head()

,income
0,0
1,0
2,0
3,0
4,0


In [111]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

pd.DataFrame(
    {
    'var': ['X_train', 'X_test', 'Y_train', 'Y_test'],
    'shape': [var.shape for var in (X_train, X_test, Y_train, Y_test)]
    })

,var,shape
0,X_train,"(22792, 14)"
1,X_test,"(9769, 14)"
2,Y_train,"(22792,)"
3,Y_test,"(9769,)"


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

> Setting a specific 'random_state' helps us get the same split of our data between runs.  
> This is useful for getting consistent results when we run our code multiple times, making it easier to understand the changes we make between experiments.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [112]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer

# Split numerical and categorical columns:
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass','education','marital-status','occupation', 'relationship', 'race', 'sex','native-country']

# Pipelines
num_transformer = Pipeline(steps=[("knnimputer", KNNImputer(n_neighbors=7, weights='distance')), ("robust_scaler", RobustScaler())])

cat_transformer = Pipeline(
    steps=[
        ("simple_imputer", SimpleImputer(strategy='most_frequent')),
        ("onehot", OneHotEncoder(drop='if_binary', handle_unknown='infrequent_if_exist')),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_transforms", num_transformer, num_features),
        ("cat_transforms", cat_transformer, cat_features),
    ]
)

display(preprocessor)

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knnimputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [113]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(
    [
        ('preprocessing', preprocessor),
        ('classifier', RandomForestClassifier())
    ]
)
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [114]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc','accuracy', 'balanced_accuracy']
cv = cross_validate(pipe, X_train, Y_train, cv=5, scoring = scoring, return_train_score = True)
pd.DataFrame(cv)

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
0,6.832557,0.162827,-0.356380,-0.081110,0.904480,1.0,0.848870,0.999945,0.773633,0.999887
1,6.720362,0.159986,-0.374800,-0.080889,0.901923,1.0,0.844483,1.000000,0.762748,1.000000
2,6.776740,0.156270,-0.348832,-0.081584,0.902256,1.0,0.853883,1.000000,0.776795,1.000000
3,6.788064,0.156515,-0.337037,-0.081886,0.907277,1.0,0.858929,0.999945,0.781664,0.999964
4,6.704826,0.154962,-0.385925,-0.080867,0.903455,1.0,0.858491,0.999890,0.779221,0.999774


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [115]:
cv_df = pd.DataFrame(cv)
cv_df.sort_values(by='test_neg_log_loss')

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
4,6.704826,0.154962,-0.385925,-0.080867,0.903455,1.0,0.858491,0.999890,0.779221,0.999774
1,6.720362,0.159986,-0.374800,-0.080889,0.901923,1.0,0.844483,1.000000,0.762748,1.000000
0,6.832557,0.162827,-0.356380,-0.081110,0.904480,1.0,0.848870,0.999945,0.773633,0.999887
2,6.776740,0.156270,-0.348832,-0.081584,0.902256,1.0,0.853883,1.000000,0.776795,1.000000
3,6.788064,0.156515,-0.337037,-0.081886,0.907277,1.0,0.858929,0.999945,0.781664,0.999964


Calculate the mean of each metric. 

In [116]:
pd.DataFrame(cv_df.mean(), columns=['Mean'])

,Mean
fit_time,6.764510
score_time,0.158112
test_neg_log_loss,-0.360595
train_neg_log_loss,-0.081267
test_roc_auc,0.903878
train_roc_auc,1.000000
test_accuracy,0.852931
train_accuracy,0.999956
test_balanced_accuracy,0.774812
train_balanced_accuracy,0.999925


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [117]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

pipeline.fit(X_train, Y_train)
Y_predict = pipeline.predict(X_test)
Y_pred_proba = pipeline.predict_proba(X_test)

performance_metrics = {
        'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
        'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
        'accuracy': accuracy_score(Y_test, Y_predict),
        'balanced_accuracy': balanced_accuracy_score(Y_test, Y_predict)
    }

pd.DataFrame(performance_metrics, index=[0]).T.style.hide(axis='columns').set_caption("<h3>Performance_metrics")

neg_log_loss,-0.401623
roc_auc,0.900414
accuracy,0.854847
balanced_accuracy,0.775707


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

> Here, we are choosing an arbitrary threshold of $50,000 in income as the most relevant indicator.   
> By making it a binary target, we simplify the application of certain models while also facilitating a clearer understanding and explanation of their results.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.